# Export darknet as onnx

## Install dependencies

In [ ]:
!apt-get install -y libopencv-dev libprotobuf-dev

## Install darknet

In [ ]:
%env DARKNET_VERSION=5.0.167
!sudo dpkg -i "/packages/darknet-${DARKNET_VERSION}-Linux$(if [[ $USE_GPU == 'OFF' ]]; then echo '-cpu'; fi).deb"

## Download people-r-people and export as onnx

In [ ]:
%cd /content
!wget --no-clobber https://www.ccoderun.ca/programming/2025-01-20_People-R-People/download/people-r-people_2025-05-05.zip
!unzip -o people-r-people_2025-05-05.zip -d people-r-people_2025-05-05

In [ ]:
%cd /content/people-r-people_2025-05-05
!darknet_onnx_export people-r-people.cfg  people-r-people.weights people-r-people.names

## Visualize Model

In [ ]:
%pip install netron

In [ ]:
import IPython

import netron

port = 8081
model_path = "/content/people-r-people_2025-05-05/people-r-people.onnx"
netron.start(model_path, 8081, browse=False)

IPython.display.IFrame(f"http://localhost:{port}", width=1000, height=1000)

In [ ]:
%pip install onnxruntime numpy opencv-python

In [ ]:
%cd /content

In [ ]:
import os
import cv2
import onnxruntime as ort
import matplotlib.pyplot as plt

from importlib import reload
import functions.parse_darknet_cfg as parse_darknet_cfg_mod

reload(parse_darknet_cfg_mod)
import functions.preprocess_image as preprocess_image_mod

reload(preprocess_image_mod)
import functions.postprocess_output as postprocess_output_mod

reload(postprocess_output_mod)
import functions.visualize_detections as visualize_detections_mod

reload(visualize_detections_mod)

DARKNET_MODEL_BASE_NAME = "people-r-people_2025-05-05/people-r-people"
NAMES_FILE = f"{DARKNET_MODEL_BASE_NAME}.names"
CFG_FILE = f"{DARKNET_MODEL_BASE_NAME}.cfg"
CONF_THRESHOLD = 0.25
IOU_THRESHOLD = 0.45
IMAGES_PATH = "people-r-people_2025-05-05/sample_images"

# Load class names
with open(NAMES_FILE, "r", encoding="utf-8") as f:
    class_names = [line.strip() for line in f if line.strip()]

num_classes = len(class_names)

print(f"✅ Loaded {num_classes} classes")

# Load darknet cfg
net, layers = parse_darknet_cfg_mod.parse_darknet_cfg(CFG_FILE)
input_width = net["width"]
input_height = net["height"]
yolo_layers = [layer for layer in layers if layer["type"] == "yolo"]

session = ort.InferenceSession(
    f"{DARKNET_MODEL_BASE_NAME}.onnx", providers=["CPUExecutionProvider"]
)

for image_name in [img for img in os.listdir(IMAGES_PATH) if img.endswith(".jpg")]:
    image_path = f"{IMAGES_PATH}/{image_name}"

    print(f"Processing image {image_path}")

    image = cv2.imread(image_path)
    input_tensor, scale, padding = preprocess_image_mod.preprocess_image(
        input_width, input_height, image
    )

    print("Running inference...")
    input_name = session.get_inputs()[0].name
    output_tensor = session.run(None, {input_name: input_tensor})

    print("Postprocessing...")
    detections = postprocess_output_mod.postprocess_output(
        output_tensor,
        yolo_layers_cfg=yolo_layers,
        input_size=(input_width, input_height),
        conf_threshold=CONF_THRESHOLD,
        iou_threshold=IOU_THRESHOLD,
        scale=scale,
        padding=padding,
        class_names=class_names,
    )

    print("Visualize...")
    visualized = visualize_detections_mod.visualize_detections(
        image=image, detections=detections[0], class_names=class_names
    )

    plt.imshow(cv2.cvtColor(visualized, cv2.COLOR_BGR2RGB), cmap="gray")
    plt.axis("off")
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
    plt.show()